In [460]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import xgboost as xgb
import datetime
from sklearn.linear_model import LogisticRegression
import pickle

In [461]:
df_with_label=pd.read_csv('data/df_with_label.csv')
df_with_label.drop(columns=['Unnamed: 0'],inplace=True)

In [462]:
df_mv=pd.read_csv('data/market_value.csv')
industries_class=pd.read_csv('data/industries_class.csv')
industries_code=pd.read_csv('data/industries_code.csv')

In [463]:
df_with_label['label_time']=pd.to_datetime((df_with_label['label_time']).apply(str))

In [464]:
df_with_label.head()

,trade_dt_x,wind_code,ep_ttm,bp_lyr,sp_ttm,cfp_ncf_ttm,cfp_ocf_ttm,cfp_fcff_ttm,ortoev_ttm,peg,...,beta_60D_szzz,beta_120D_szzz,beta_240D_szzz,beta_480D_szzz,updatetime,trade_dt_y,s_mq_pctchange,s_mq_amount,label,label_time
0,2015-01-30,000001.SZ,0.120824,0.796337,0.436667,0.090356,-0.089589,NaN,0.397811,28.167006,...,1.357957,1.313219,1.339941,1.574382,2019-05-23 01:48:00.000,2015-02-27,0.4304,1.804355e+06,2,2015-01-30
1,2015-01-30,000002.SZ,0.106428,0.542133,0.932968,0.081991,0.215316,0.043625,0.878831,71.756337,...,1.330106,1.297934,1.226921,1.308739,2019-05-23 01:48:00.000,2015-02-27,-2.8202,3.287976e+06,0,2015-01-30
2,2015-01-30,000004.SZ,-0.005140,0.048630,0.046602,-0.010674,0.001702,-0.007284,0.047010,59.679834,...,0.367000,0.331869,0.453777,0.558445,2019-05-23 01:48:00.000,2015-02-27,7.8996,4.136355e+04,6,2015-01-30
3,2015-01-30,000005.SZ,-0.012082,0.158174,0.014907,0.000598,-0.030551,0.086292,0.015112,10.260072,...,0.311863,0.295469,0.554438,0.811105,2019-05-23 01:48:00.000,2015-02-27,0.0000,0.000000e+00,1,2015-01-30
4,2015-01-30,000006.SZ,0.065695,0.446514,0.415537,-0.023423,-0.107033,0.079987,0.376103,-163.776036,...,0.743068,0.790482,0.879827,1.056388,2019-05-23 01:48:00.000,2015-02-27,1.0795,1.422369e+05,2,2015-01-30


# 1. 剔除当日涨跌停的股票

In [465]:
df_mv['TRADE_DT']=pd.to_datetime((df_mv['TRADE_DT']).apply(str))

In [466]:
df_mv.head()

,S_INFO_WINDCODE,TRADE_DT,S_DQ_MV,UP_DOWN_LIMIT_STATUS
0,600230.SH,2014-08-29,3.030139e+05,0.0
1,600850.SH,2014-08-29,4.612720e+05,0.0
2,600516.SH,2014-08-29,1.903111e+06,0.0
3,300281.SZ,2014-08-29,9.712500e+04,0.0
4,300058.SZ,2014-08-29,1.225476e+06,0.0


In [467]:
df_with_status=pd.merge(df_with_label, df_mv, how='inner', left_on=['label_time','wind_code'],right_on=['TRADE_DT','S_INFO_WINDCODE'])

In [468]:
df_with_status=df_with_status[df_with_status['UP_DOWN_LIMIT_STATUS']==0]

# 2. 增加行业代码因子

In [469]:
temp_df=pd.merge(df_with_status, industries_class, how='inner', left_on=['wind_code'],right_on=['WIND_CODE'])

In [470]:
temp_df['ENTRY_DT']=pd.to_datetime(temp_df['ENTRY_DT'].apply(str))

In [471]:
temp_df['REMOVE_DT'].fillna(20191231,inplace=True)

In [472]:
temp_df['REMOVE_DT']=pd.to_datetime(temp_df['REMOVE_DT'].apply(int).apply(str))

In [473]:
temp_df.shape

(251815, 195)

In [474]:
df_factors=temp_df[np.array(temp_df['ENTRY_DT']<= temp_df['label_time']) & np.array(temp_df['label_time']<=temp_df['REMOVE_DT'] )]

In [475]:
df_factors.reset_index(drop=True,inplace=True)

In [476]:
df_mv.columns

Index(['S_INFO_WINDCODE', 'TRADE_DT', 'S_DQ_MV', 'UP_DOWN_LIMIT_STATUS'], dtype='object')

In [477]:
industries_class.columns

Index(['OBJECT_ID', 'WIND_CODE', 'S_INFO_WINDCODE', 'CITICS_IND_CODE',
       'ENTRY_DT', 'REMOVE_DT', 'CUR_SIGN', 'OPDATE', 'OPMODE'],
      dtype='object')

In [478]:
df_factors

,trade_dt_x,wind_code,ep_ttm,bp_lyr,sp_ttm,cfp_ncf_ttm,cfp_ocf_ttm,cfp_fcff_ttm,ortoev_ttm,peg,...,UP_DOWN_LIMIT_STATUS,OBJECT_ID,WIND_CODE,S_INFO_WINDCODE_y,CITICS_IND_CODE,ENTRY_DT,REMOVE_DT,CUR_SIGN,OPDATE,OPMODE
0,2015-01-30,000001.SZ,0.120824,0.796337,0.436667,0.090356,-0.089589,NaN,0.397811,28.167006,...,0.0,{A52B5E9E-CF90-4373-9B8E-321874AA5C0B},000001.SZ,000001.SZ,b10l020100,2003-01-01,2019-12-31,1,2014/2/10 15:43:16.000,0
1,2015-02-27,000001.SZ,0.120306,0.792921,0.434794,0.089968,-0.089205,NaN,0.396256,28.288329,...,0.0,{A52B5E9E-CF90-4373-9B8E-321874AA5C0B},000001.SZ,000001.SZ,b10l020100,2003-01-01,2019-12-31,1,2014/2/10 15:43:16.000,0
2,2015-03-31,000001.SZ,0.110047,0.727729,0.407948,0.013071,0.140717,NaN,0.375509,30.278957,...,0.0,{A52B5E9E-CF90-4373-9B8E-321874AA5C0B},000001.SZ,000001.SZ,b10l020100,2003-01-01,2019-12-31,1,2014/2/10 15:43:16.000,0
3,2015-04-30,000001.SZ,0.089000,0.597078,0.340582,-0.227067,-0.101684,NaN,0.318929,50.963154,...,0.0,{A52B5E9E-CF90-4373-9B8E-321874AA5C0B},000001.SZ,000001.SZ,b10l020100,2003-01-01,2019-12-31,1,2014/2/10 15:43:16.000,0
4,2015-05-29,000001.SZ,0.092957,0.623624,0.355725,-0.237162,-0.106205,NaN,0.332170,48.793794,...,0.0,{A52B5E9E-CF90-4373-9B8E-321874AA5C0B},000001.SZ,000001.SZ,b10l020100,2003-01-01,2019-12-31,1,2014/2/10 15:43:16.000,0
5,2015-06-30,000001.SZ,0.097944,0.657079,0.374807,-0.249884,-0.111902,NaN,0.348750,46.309515,...,0.0,{A52B5E9E-CF90-4373-9B8E-321874AA5C0B},000001.SZ,000001.SZ,b10l020100,2003-01-01,2019-12-31,1,2014/2/10 15:43:16.000,0
6,2015-07-31,000001.SZ,0.115219,0.772971,0.440914,-0.293958,-0.131639,NaN,0.405291,39.366273,...,0.0,{A52B5E9E-CF90-4373-9B8E-321874AA5C0B},000001.SZ,000001.SZ,b10l020100,2003-01-01,2019-12-31,1,2014/2/10 15:43:16.000,0
7,2015-08-31,000001.SZ,0.134567,0.952543,0.538198,0.984829,0.974886,NaN,0.490101,37.275793,...,0.0,{A52B5E9E-CF90-4373-9B8E-321874AA5C0B},000001.SZ,000001.SZ,b10l020100,2003-01-01,2019-12-31,1,2014/2/10 15:43:16.000,0
8,2015-09-30,000001.SZ,0.142007,1.005210,0.567956,1.039281,1.028788,NaN,0.514656,35.322770,...,0.0,{A52B5E9E-CF90-4373-9B8E-321874AA5C0B},000001.SZ,000001.SZ,b10l020100,2003-01-01,2019-12-31,1,2014/2/10 15:43:16.000,0
9,2015-10-30,000001.SZ,0.134411,0.966714,0.553122,1.335008,0.753107,NaN,0.504843,54.624478,...,0.0,{A52B5E9E-CF90-4373-9B8E-321874AA5C0B},000001.SZ,000001.SZ,b10l020100,2003-01-01,2019-12-31,1,2014/2/10 15:43:16.000,0


In [479]:
stock_info=df_factors[['trade_dt_x','wind_code','CITICS_IND_CODE']]

In [480]:
stock_info.head()

,trade_dt_x,wind_code,CITICS_IND_CODE
0,2015-01-30,000001.SZ,b10l020100
1,2015-02-27,000001.SZ,b10l020100
2,2015-03-31,000001.SZ,b10l020100
3,2015-04-30,000001.SZ,b10l020100
4,2015-05-29,000001.SZ,b10l020100


In [481]:
stock_info['trade_dt_x']=pd.to_datetime(stock_info['trade_dt_x'])

C:\Users\surface\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [482]:
df_factors.drop(columns=['OBJECT_ID', 'WIND_CODE', 'S_INFO_WINDCODE_x', 
       'ENTRY_DT', 'REMOVE_DT', 'CUR_SIGN', 'OPDATE', 'OPMODE','S_INFO_WINDCODE_y', 'TRADE_DT', 'S_DQ_MV', 'UP_DOWN_LIMIT_STATUS'],inplace=True)

In [483]:
df_factors.drop(columns=['trade_dt_x','wind_code','updatetime','trade_dt_y','s_mq_pctchange','s_mq_amount'],inplace=True)

# 3.删除因子间相关性过高的项

In [484]:
correlation=df_factors.corr()
corr_map=correlation.where((correlation>0.8) | (correlation < -0.8))
corr_li=[]
for i in correlation.index:
    for j in correlation.columns:
        if ~np.isnan(corr_map[i][j]) and corr_map[i][j]!=1 and (j,i) not in corr_li:
            corr_li.append((i,j))
degree_dict=dict()
for i in corr_li:
    for a in i:
        degree_dict[a]=degree_dict.get(a,0)+1
record_factor=[]
def find_max_key(degree_dict):
    return list(degree_dict.keys())[list(degree_dict.values()).index((max(degree_dict.values())))]

In [485]:
sum(degree_dict.values())

214

In [486]:
def delete_factor():
    while sum(degree_dict.values())!=0:
        max_key=find_max_key(degree_dict)
        degree_dict.pop(max_key)
        record_factor.append(max_key)
        temp_li=[]
        for index,i in enumerate(corr_li):
            if max_key in i:
                temp_li.append(i)
                for a in i:
                    if a!=max_key:
                        degree_dict[a]=degree_dict.get(a)-1
        for i in temp_li:
            corr_li.remove(i)

In [487]:
delete_factor()

In [488]:
record_factor

['vol_120D',
 'finaexpensetogr_ttm2',
 'ivol_capm_120D_szzz',
 'threecosttosales_ttm',
 'ivol_capm_240D_zz500',
 'beta_120D_zz500',
 'ivol_capm_60D_szzz',
 'ivol_capm_240D_szzz',
 'beta_240D_zz500',
 'debttoequity',
 'operateprofitmargin_ttm',
 'vol_20D',
 'roa_avg_ttm',
 'netprofitmargin_ttm',
 'operateexpensetogr_ttm2',
 'net_profit_yoy_ttm',
 'net_profit_qoq_ttm',
 'net_profit_3y_ttm',
 'mom_180D',
 'mom_tw_60D',
 'turnover_20D',
 'ivol_capm_60D_zz500',
 'beta_60D_szzz',
 'beta_240D_szzz',
 'roe_avg',
 'roe_diluted',
 'roa_diluted_ttm',
 'grossprofitmargin_qoq_ttm',
 'nonoperateprofittoebt_ttm',
 'eps_ttm',
 'current',
 'oper_profit_5y_ttm',
 'net_profit_yoy',
 'net_profit_qoq',
 'roe_yoy_ttm',
 'roe_qoq_ttm',
 'roe_3y_ttm',
 'grossprofitmargin_yoy',
 'roe_5y_ttm',
 'mom_240D',
 'mom_tw_120D',
 'turnover_60D',
 'turnbias_20Dto60D',
 'ivol_capm_20D_zz500',
 'volbias_20Dto60D',
 'ivol_capm_480D_zz500',
 'dif',
 's_val_mv',
 'ln_mv',
 'beta_20D_zz500',
 'beta_480D_zz500']

In [489]:
df_factors.drop(columns=record_factor,inplace=True)

In [490]:
df_factors.shape

(179123, 126)

In [491]:
df_factors['CITICS_IND_CODE'].unique()

array(['b10l020100', 'b10n010100', 'b10i010200', 'b10n010200',
       'b104020100', 'b103030100', 'b10n020100', 'b10f020100',
       'b10o010200', 'b10a040100', 'b10t010100', 'b10a020300',
       'b107020100', 'b107010100', 'b108030200', 'b108020100',
       'b10g020100', 'b10a040300', 'b10j020400', 'b10p030100',
       'b10p020100', 'b108010100', 'b10d040100', 'b109020300',
       'b10e010300', 'b104010100', 'b10i040100', 'b10d030100',
       'b10q020300', 'b10k020100', 'b10r020100', 'b10r010100',
       'b10r030200', 'b10b010100', 'b10s010500', 'b10a030300',
       'b101020100', 'b103020200', 'b10e010200', 'b10k010300',
       'b10q020600', 'b10n010300', 'b10q020700', 'b10i030100',
       'b10o010100', 'b10o040200', 'b101020400', 'b10o040100',
       'b10i040300', 'b10e020100', 'b106010100', 'b10s010200',
       'b10a010100', 'b10h040100', 'b10r020200', 'b10m010100',
       'b101020500', 'b10g010100', 'b10b020200', 'b104020300',
       'b105020300', 'b106010200', 'b105020200', 'b10a0

In [492]:
industries_dict=dict(zip(df_factors['CITICS_IND_CODE'].unique(),[i for i in range(188)]))

In [493]:
li=[]
for i in df_factors['CITICS_IND_CODE']:
    li.append(industries_dict[i])

In [494]:
df_factors['CITICS_IND_CODE']=np.array(li)

C:\Users\surface\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


# 4.训练集及测试集划分

In [495]:
training_data=df_factors[df_factors['label_time']<datetime.datetime(2019,1,1)]
test_data=df_factors[df_factors['label_time']>=datetime.datetime(2019,1,1)]
training_data.drop(columns=['label_time'],inplace=True)
test_data.drop(columns=['label_time'],inplace=True)
X_train=training_data.loc[:,training_data.columns!='label']
y_train=training_data['label'].values
X_test=test_data.loc[:,test_data.columns!='label']
y_test=test_data['label'].values

In [496]:
stock_info_2019=stock_info[stock_info['trade_dt_x']>=datetime.datetime(2019,1,1)]

In [497]:
stock_info_2019.to_csv('xgboost/data/industries/stock_info_2019.csv')

In [498]:
len(y_test)==len(stock_info_2019)==len(X_test)

True

# 5. 数据管道

In [346]:
## process pipeline
process_scale = make_column_transformer(
    (make_pipeline(SimpleImputer(strategy='mean'), StandardScaler()), 
     X_train.columns),
    remainder='passthrough')

# 6. Xgboost分类器

In [292]:
params = {
    'booster': 'gbtree',
    'objective': 'multi:softmax',  
    'num_class': 10,               
    'gamma': 0.1,                  
    'lambda': 2,                   
    'subsample': 0.7,              
    'colsample_bytree': 0.7,      
    'min_child_weight': 3,
    'silent': 1,                   
    'eta': 0.007,                  
    'nthread': 8                  
}

In [297]:
param_grid = {'xgbclassifier__max_depth':range(3, 10, 3)}
mean_pipe_xgb = make_pipeline(process_scale,
                          xgb.XGBClassifier(params))
grid_xgb = GridSearchCV(mean_pipe_xgb, param_grid, cv=5, scoring='accuracy',verbose=2)
grid_xgb.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] xgbclassifier__max_depth=3 ......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ....................... xgbclassifier__max_depth=3, total=15.9min
[CV] xgbclassifier__max_depth=3 ......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 15.9min remaining:    0.0s


[CV] ....................... xgbclassifier__max_depth=3, total=15.8min
[CV] xgbclassifier__max_depth=3 ......................................
[CV] ....................... xgbclassifier__max_depth=3, total=15.7min
[CV] xgbclassifier__max_depth=3 ......................................
[CV] ....................... xgbclassifier__max_depth=3, total=15.2min
[CV] xgbclassifier__max_depth=3 ......................................
[CV] ....................... xgbclassifier__max_depth=3, total=15.6min
[CV] xgbclassifier__max_depth=6 ......................................
[CV] ....................... xgbclassifier__max_depth=6, total=29.0min
[CV] xgbclassifier__max_depth=6 ......................................
[CV] ....................... xgbclassifier__max_depth=6, total=29.8min
[CV] xgbclassifier__max_depth=6 ......................................
[CV] ....................... xgbclassifier__max_depth=6, total=29.4min
[CV] xgbclassifier__max_depth=6 ......................................
[CV] .

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed: 454.0min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('columntransformer',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='passthrough',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('pipeline',
                                                                         Pipeline(memory=None,
                                                                                  steps=[('simpleimputer',
                                                                                          SimpleImputer(add_indicator=False,
                                                                                                        copy=Tru

In [302]:
pickle.dump(grid_xgb, open("xgboost/industries_xgboost.pickle.dat", "wb"))

In [304]:
model=pickle.load(open("xgboost/industries_xgboost.pickle.dat","rb"))

In [499]:
model.score(X_test, y_test)

0.14070991847826086

In [500]:
y_pred=model.predict(X_test)

In [501]:
confusion_matrix(y_test,y_pred)

array([[1560,  232,  191,  104,  153,  164,  112,  251,  256,  307],
       [ 983,  305,  235,  208,  348,  375,  197,  365,  245,  272],
       [ 763,  262,  282,  267,  375,  509,  252,  409,  216,  236],
       [ 607,  272,  252,  268,  473,  606,  273,  430,  199,  199],
       [ 552,  246,  247,  299,  476,  613,  294,  421,  202,  239],
       [ 469,  249,  247,  292,  492,  662,  319,  434,  206,  229],
       [ 486,  239,  232,  281,  505,  610,  281,  432,  260,  252],
       [ 593,  224,  276,  265,  430,  537,  232,  465,  242,  284],
       [ 704,  301,  248,  235,  314,  446,  211,  425,  298,  365],
       [ 996,  313,  272,  213,  301,  298,  155,  278,  254,  374]],
      dtype=int64)

In [505]:
stock_info_2019.reset_index(drop=True,inplace=True)

In [516]:
stock_info_2019.loc[np.where(y_pred==9)].to_csv('xgboost/data/industries/portfolio_9.csv')
stock_info_2019.loc[np.where(y_pred==8)].to_csv('xgboost/data/industries/portfolio_8.csv')
stock_info_2019.loc[np.where(y_pred==7)].to_csv('xgboost/data/industries/portfolio_7.csv')
stock_info_2019.loc[np.where(y_pred==6)].to_csv('xgboost/data/industries/portfolio_6.csv')
stock_info_2019.loc[np.where(y_pred==5)].to_csv('xgboost/data/industries/portfolio_5.csv')
stock_info_2019.loc[np.where(y_pred==4)].to_csv('xgboost/data/industries/portfolio_4.csv')
stock_info_2019.loc[np.where(y_pred==3)].to_csv('xgboost/data/industries/portfolio_3.csv')
stock_info_2019.loc[np.where(y_pred==2)].to_csv('xgboost/data/industries/portfolio_2.csv')
stock_info_2019.loc[np.where(y_pred==1)].to_csv('xgboost/data/industries/portfolio_1.csv')
stock_info_2019.loc[np.where(y_pred==0)].to_csv('xgboost/data/industries/portfolio_0.csv')

In [518]:
np.savetxt("xgboost/confusion.csv",confusion_matrix(y_test,y_pred),delimiter=",")